##### Copyright 2019 The IREE Authors

In [1]:
#@title Licensed under the Apache License v2.0 with LLVM Exceptions.
# See https://llvm.org/LICENSE.txt for license information.
# SPDX-License-Identifier: Apache-2.0 WITH LLVM-exception

# Low Level Invoke Function

This notebook shows off some concepts of the low level IREE python bindings.

In [2]:
!python -m pip install iree-compiler iree-runtime -f https://iree.dev/pip-release-links.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://iree.dev/pip-release-links.html
     |████████████████████████████████| 49.7 MB 99 kB/s 
     |████████████████████████████████| 2.3 MB 40.7 MB/s 


In [1]:
import numpy as np
import inspect
import sys
sys.path.insert(0, "/home/v-yinuoliu/code/iree/iree-build/compiler/bindings/python")
# sys.path.insert(0, "/home/v-yinuoliu/miniconda3/envs/iree_new/lib/python3.10/site-packages")
from iree.compiler import compile_str
import iree.compiler
import iree.runtime as ireert
print(iree.compiler.__file__)
print(ireert.__file__)
print(inspect.getfile(compile_str))

/home/v-yinuoliu/miniconda3/envs/iree_new/lib/python3.10/site-packages/iree/compiler/__init__.py
/home/v-yinuoliu/miniconda3/envs/iree_new/lib/python3.10/site-packages/iree/runtime/__init__.py
/home/v-yinuoliu/miniconda3/envs/iree_new/lib/python3.10/site-packages/iree/compiler/tools/core.py


In [2]:
# Compile a module.
SIMPLE_MUL_ASM = """
  module @arithmetic {
    func.func @simple_mul(%arg0: tensor<4xf32>, %arg1: tensor<4xf32>) -> tensor<4xf32> {
      %0 = arith.mulf %arg0, %arg1 : tensor<4xf32>
      return %0 : tensor<4xf32>
    } 
  }
"""

# Compile using the vmvx (reference) target:
print(inspect.getfile(compile_str))
compiled_flatbuffer = compile_str(SIMPLE_MUL_ASM, target_backends=["vmvx"])

/home/v-yinuoliu/miniconda3/envs/iree_new/lib/python3.10/site-packages/iree/compiler/tools/core.py


In [3]:
# Register the module with a runtime context.
# Use the "local-task" CPU driver, which can load the vmvx executable:
config = ireert.Config("local-task")
ctx = ireert.SystemContext(config=config)
vm_module = ireert.VmModule.from_flatbuffer(ctx.instance, compiled_flatbuffer)
ctx.add_vm_module(vm_module)

# Invoke the function and print the result.
print("INVOKE simple_mul")
arg0 = np.array([1., 2., 3., 4.], dtype=np.float32)
arg1 = np.array([4., 5., 6., 7.], dtype=np.float32)
f = ctx.modules.arithmetic["simple_mul"]
results = f(arg0, arg1).to_host()
print("Results:", results)

INVOKE simple_mul
Results: [ 4. 10. 18. 28.]
